# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846847366850                   -0.70    4.5         
  2   -7.852317325247       -2.26       -1.53    1.0   16.8ms
  3   -7.852614631083       -3.53       -2.56    1.5   18.4ms
  4   -7.852645997334       -4.50       -2.90    2.8   23.3ms
  5   -7.852646522465       -6.28       -3.21    1.0   16.9ms
  6   -7.852646679645       -6.80       -4.04    1.0   17.0ms
  7   -7.852646686375       -8.17       -5.24    1.5   19.5ms
  8   -7.852646686725       -9.46       -5.57    2.5   22.5ms
  9   -7.852646686730      -11.39       -6.13    1.5   19.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846863236528                   -0.70           4.5         
  2   -7.852551026559       -2.25       -1.63   0.80    2.0    213ms
  3   -7.852638161915       -4.06       -2.74   0.80    1.0   16.0ms
  4   -7.852646537797       -5.08       -3.31   0.80    2.0   19.7ms
  5   -7.852646672114       -6.87       -4.21   0.80    1.0   15.8ms
  6   -7.852646686477       -7.84       -4.73   0.80    1.8   19.4ms
  7   -7.852646686714       -9.63       -5.54   0.80    1.8   18.1ms
  8   -7.852646686730      -10.80       -6.76   0.80    2.0   20.0ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.376353e+01     3.284065e+00
 * time: 0.04589986801147461
     1     1.637836e+00     1.747248e+00
 * time: 0.22224092483520508
     2    -1.499738e+00     2.074339e+00
 * time: 0.24021482467651367
     3    -3.945209e+00     1.792429e+00
 * time: 0.26591992378234863
     4    -5.333824e+00     1.552890e+00
 * time: 0.2916560173034668
     5    -6.997194e+00     1.101702e+00
 * time: 0.3182358741760254
     6    -7.165407e+00     1.330151e+00
 * time: 0.3363230228424072
     7    -7.649890e+00     1.147139e+00
 * time: 0.35437488555908203
     8    -7.701094e+00     1.626469e+00
 * time: 0.3722670078277588
     9    -7.730534e+00     1.420209e+00
 * time: 0.3901050090789795
    10    -7.771774e+00     9.308762e-01
 * time: 0.41590285301208496
    11    -7.802002e+00     5.451271e-01
 * time: 0.441756010055542
    12    -7.830137e+00     2.492478e-01
 * time: 0.4596400260925293
    13    -7.838693e+00     4.475729e-02
 * time: 0.47754

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846905940231                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645918879                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.54s
  3   -7.852646686730      -13.32       -7.25    119ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.386834283588283e-7
|ρ_newton - ρ_scfv| = 3.1770256381208704e-7
|ρ_newton - ρ_dm|   = 1.087710936559735e-9
